# Иницилизация

In [436]:
import numpy as np

maps=np.load("./Task19.npy")
print("Исходная матрица:","\n")
maps = np.where(maps!=0,maps,1)
print(maps)

Исходная матрица: 

[[ 1 57 55  4 26 84 72 14 11 55]
 [57  1 73 65 38 97 41 13 39 57]
 [55 73  1 19 36 25 63  2 87 81]
 [ 4 65 19  1 46 60 80 21 41 84]
 [26 38 36 46  1 17  1 66 60 52]
 [84 97 25 60 17  1 50 97 88 45]
 [72 41 63 80  1 50  1  5 79 46]
 [14 13  2 21 66 97  5  1  3 94]
 [11 39 87 41 60 88 79  3  1 46]
 [55 57 81 84 52 45 46 94 46  1]]


# Выбор маршрута, рандомом

In [26]:
import random
import time        

def getRandomTrace(maps):
    trace=[1] 
    pathLenght = 0
    cities=[1,2,3,4,5,6,7,8,9,10]
    cnt=1
    gratedMass=[int(1e10)]
    start=time.perf_counter()
    while cnt !=  np.math.factorial(10):
        cnt+=1
        if len(cities)>0:
            nextStep=random.choice(cities)
            pathLenght+=maps.item(trace[-1]-1,nextStep-1)
            trace.append(nextStep)
            cities.remove(nextStep)
            """print("Длина пути: ",pathLenght)
            print("Пройденный маршрут: ",trace)
            print("Оставшиеся города: ",cities)"""
        else:
            pathLenght+=maps.item(trace[-1]-1,0)
            if pathLenght<gratedMass[-1]:
                gratedMass.append(pathLenght)
            gratedMass.sort()
            if len(gratedMass)>5:
                gratedMass.pop()
            #print(pathLenght)
            pathLenght = 0
            trace=[1]
            cities=[2,3,4,5,6,7,8,9,10]
    end = time.perf_counter()
    print("Затраченное время:",end-start)
    print("Массив минимальных путей: ",gratedMass)
    
getRandomTrace(maps)    

Затраченное время: 4.002264388000185
Массив минимальных путей:  [198, 199, 199, 199, 200]


# Базова реализация алгоритма

In [453]:


def getP(i,pher,n,cities):
    
    alfa=10
    beta=10
    
    p=[]
    summa=0
    i-=1
    for x in cities:
        x-=1
        summa+=pher.item(i,x)**alfa*n.item(i,x)**beta
#     print("Summa: ",summa)
#     print("Cities: ",cities)
    for x in cities:
        try:
            x-=1
            p.append((pher.item(i,x)**alfa*n.item(i,x)**beta)/(summa))
        except ZeroDivisionError:
            p.append(1)
    return p

def getNextCity(p):
    point=np.random.randint(0,100)/100
#     print("probability: ",p)
#     print("Summa: ",summa)
#     print("Point: ",point)
    delta=0
    for i in p:
        delta+=i
        if delta>point:
            cityIndex = p.index(i)
#             print("Delta: ",delta)
#             print("NextCity: ",cityIndex)
            break
    return cityIndex
    
    

def ANT(numberOfAnts,numberOfIterations):
    import numpy as np
    import time 
    print("Процесс начат")
    
    minLenght=int(1e10)
    start=time.perf_counter()
    
#     trace=[1]
#     cities=[2,5,4,3,7,6,8,9,10]
    
    trace=[np.random.randint(1,10)]
    cities=[1,2,5,4,3,7,6,8,9,10]
    cities.remove(trace[-1])

    pheromons=np.full((10,10),0.1)
    n=np.where(maps!=1,1/maps,1)
    

    Q=1000;
    speed=0.1
    
    for it in range(0,numberOfIterations):
#         print("Номер итерации:",it)
        paths=[]
        for a in range(1,numberOfAnts+1):
#             print("Номер муравья:",a)
            while len(cities)>0:
                probability=getP(trace[-1],pheromons,n,cities)
                indexOfNextCity=getNextCity(probability)
#                 print("Next city: ",cities[indexOfNextCity])
                trace.append(cities[indexOfNextCity])
                cities.pop(indexOfNextCity)
#                 print("Пройденный маршрут: ",trace)
#                 print("Оставшиеся города: ",cities)
            paths.append(trace)
#             trace=[1]
#             cities=[2,5,4,3,7,6,8,9,10]
            trace=[np.random.randint(1,10)]
            cities=[1,2,5,4,3,7,6,8,9,10]
            cities.remove(trace[-1])
#         for p in paths:
#             print(p)
        
        evaporation=lambda x: speed*x
        pheromons=np.vectorize(evaporation)(pheromons)
        
        #Рассчет маршрута
        lenghts=[]
        for p in paths:
            lenght=0
            counter=-1
#             print("-------")
            for l in p:
                counter+=1
                if (counter)<9:
#                     print(l,p[counter+1])
                    lenght+=maps.item(l-1,p[counter+1]-1)
                else:
#                     print(l,p[0])
                    lenght+=maps.item(l-1,p[0]-1)
                
            lenghts.append(lenght)
#         print(lenghts)
        
        #Обновление феормона на путях
        for l in lenghts:
            for p in paths:
                counter=-1
                for m in p:
                    counter+=1
                    if (counter)<9:
                        np.add.at(pheromons,(m-1,p[counter+1]-1),Q/l)
                        np.add.at(pheromons,(p[counter+1]-1,m-1),Q/l)
                    else:
                        np.add.at(pheromons,(p[0]-1,m-1),Q/l)
                        np.add.at(pheromons,(m-1,p[0]-1),Q/l)
#         print("Феромоны: ",pheromons)
#         print(n)
        for l in lenghts:
            if l<minLenght:
                minLenght=l
        if minLenght<205:
            end=time.perf_counter()
            break
        else:
            end=time.perf_counter()
    T=(end-start)
    print("T= ",T,"c")        
    print(minLenght)
    print(lenghts)
    print(paths)
    
                                   
ANT(2,10000)                
                
            

Процесс начат
T=  3.825030562999018 c
226
[226, 226]
[[3, 8, 9, 2, 7, 5, 6, 10, 1, 4], [4, 1, 10, 6, 5, 7, 2, 9, 8, 3]]


# Элитные муравьиные системы

In [468]:


def getP(i,pher,n,cities):
    alfa=10
    beta=3
    
    p=[]
    summa=0
    i-=1
    for x in cities:
        x-=1
        summa+=pher.item(i,x)**alfa*n.item(i,x)**beta
    for x in cities:
        try:
            x-=1
            p.append((pher.item(i,x)**alfa*n.item(i,x)**beta)/(summa))
        except ZeroDivisionError:
            p.append(1)
    return p

def getNextCity(p):
    point=np.random.randint(0,100)/100
    delta=0
    for i in p:
        delta+=i
        if delta>point:
            cityIndex = p.index(i)
            break
    return cityIndex            
            
            
    
    

def ANT(numberOfAnts,numberOfIterations):
    import numpy as np
    import time 
    print("Процесс начат")
    
    minLenght=int(1e10)
    start=time.perf_counter()
    
#     trace=[1]
#     cities=[2,5,4,3,7,6,8,9,10]
    
    trace=[np.random.randint(1,10)]
    cities=[1,2,5,4,3,7,6,8,9,10]
    cities.remove(trace[-1])

    pheromons=np.full((10,10),0.1)
    n=np.where(maps!=1,1/maps,1)
    

    Q=10000;
    speed=0.1
    
    for it in range(0,numberOfIterations):
        paths=[]
        for a in range(1,numberOfAnts+1):
            while len(cities)>0:
                probability=getP(trace[-1],pheromons,n,cities)
                indexOfNextCity=getNextCity(probability)
                trace.append(cities[indexOfNextCity])
                cities.pop(indexOfNextCity)
            paths.append(trace)
#             trace=[1]
#             cities=[2,5,4,3,7,6,8,9,10]
            trace=[np.random.randint(1,10)]
            cities=[1,2,5,4,3,7,6,8,9,10]
            cities.remove(trace[-1])
#         for p in paths:

        
        evaporation=lambda x: speed*x
        pheromons=np.vectorize(evaporation)(pheromons)
        
        #Рассчет маршрута
        lenghts=[]
        bestRoute=10000
        for p in paths:
            lenght=0
            counter=-1
            for l in p:
                counter+=1
                if (counter)<9:
                    lenght+=maps.item(l-1,p[counter+1]-1)
                else:
                    lenght+=maps.item(l-1,p[0]-1)  
            lenghts.append(lenght)
        minLenght=1000   
        for l in lenghts:
            if l<minLenght:
                minLenght=l    
        #Обновление феормона на путях для элитных муравьев
        for m in paths[lenghts.index(minLenght)]:
            counter+=1
            if (counter)<9:
                np.add.at(pheromons,(m-1,p[counter+1]-1),Q/minLenght)
                np.add.at(pheromons,(p[counter+1]-1,m-1),Q/minLenght)
            else:
                np.add.at(pheromons,(p[0]-1,m-1),Q/minLenght)
                np.add.at(pheromons,(m-1,p[0]-1),Q/minLenght)
#         print("Феромоны: ",pheromons)
#         print(n)
        
        #Ограничение феормонов
        pheromons=np.where(pheromons < 0.01 ,pheromons,0.01)
        pheromons=np.where(pheromons > 6 ,pheromons,6)
        
    
        if minLenght<200:
            break
            
    end=time.perf_counter()
    T=(end-start)
    print("T= ",T,"c")        
    print(minLenght)
    print(lenghts)
    for p in paths:
        print(p)
    
                                   
ANT(10,10000)  

Процесс начат
[0.0056904997189110434, 0.0036954987661489405, 0.6865015154482906, 0.015614731228691907, 0.21585804450543697, 0.02698225556317962, 0.0036954987661489405, 0.004949259757403755, 0.037012696245788235]
[5.6888823840713214e-05, 1.822200699490405e-05, 1.0272437410869309e-05, 2.1430854934507354e-05, 0.9998779678243752, 3.4778847977863185e-06, 4.629064665853589e-06, 7.111102980089152e-06]
[0.00033342080369719524, 0.001805670958610159, 0.00024306376589525546, 0.0004977010247608279, 0.9955891851069664, 0.00025241136733274806, 0.0012785469727374327]
[0.002236244941495094, 0.0027930159851900505, 0.0006625910937763242, 0.7670320149328175, 0.2272687451652792, 7.38788144180786e-06]
[0.945675210274101, 0.021219064800061162, 0.0182628473886744, 0.0019114471838014835, 0.01293143035336184]
[0.0003452005412441431, 0.9988863099160405, 0.0003842447713576354, 0.0003842447713576354]
[0.024095237129095556, 0.9647404135556007, 0.01116434931530387]
[0.5773658330961325, 0.4226341669038674]
[1.0]
[4.

# Алгоритм муравьиной колонии

In [481]:
def getP(i,pher,n,cities,flag):
    alfa=10
    beta=3
    
    p=[]
    summa=0
    i-=1
    if flag:
        for x in cities:
            x-=1
            summa+=pher.item(i,x)**alfa*n.item(i,x)**beta
        for x in cities:
            try:
                x-=1
                p.append((pher.item(i,x)**alfa*n.item(i,x)**beta)/(summa))
            except ZeroDivisionError:
                p.append(1)
    else:
        for x in cities:
            x-=1
            p.append(pher.item(i,x))
    return p

def getNextCity(p,flag):
    if flag:
        point=np.random.randint(0,100)/100
        delta=0
        for i in p:
            delta+=i
            if delta>point:
                cityIndex = p.index(i)
                break
#         print(p)
    else:
        maxP=-1
        for i in p:
            if i>maxP:
                maxP=i
        print(p)
#         print(p.index(maxP))
        cityIndex = p.index(maxP)
    return cityIndex            
            
            
def ANT(numberOfAnts,numberOfIterations):
    import numpy as np
    import time 
    print("Процесс начат")
    
    minLenght=int(1e10)
    start=time.perf_counter()
    
#     trace=[1]
#     cities=[2,5,4,3,7,6,8,9,10]
    
    trace=[np.random.randint(1,10)]
    cities=[1,2,5,4,3,7,6,8,9,10]
    cities.remove(trace[-1])

    pheromons=np.full((10,10),0.1)
    n=np.where(maps!=1,1/maps,1)
    onesMatrix=np.ones([10,10])

    Q=10000;
    speed=0.1
    
    for it in range(0,numberOfIterations):
        paths=[]
        for a in range(1,numberOfAnts+1):
            while len(cities)>0:
                if np.random.random()>0.5:
                    probability=getP(trace[-1],pheromons,n,cities,True)
                    indexOfNextCity=getNextCity(probability,True)
                    trace.append(cities[indexOfNextCity])
                    cities.pop(indexOfNextCity)
                else:
                    probability=getP(trace[-1],pheromons,onesMatrix,cities,False)
                    indexOfNextCity=getNextCity(probability,False)
                    trace.append(cities[indexOfNextCity])
                    cities.pop(indexOfNextCity)
            paths.append(trace)
#             trace=[1]
#             cities=[2,5,4,3,7,6,8,9,10]
            trace=[np.random.randint(1,10)]
            cities=[1,2,5,4,3,7,6,8,9,10]
            cities.remove(trace[-1])

        evaporation=lambda x: speed*x
        pheromons=np.vectorize(evaporation)(pheromons)
        
        #Рассчет маршрута
        lenghts=[]
        bestRoute=10000
        for p in paths:
            lenght=0
            counter=-1
            for l in p:
                counter+=1
                if (counter)<9:
                    lenght+=maps.item(l-1,p[counter+1]-1)
                else:
                    lenght+=maps.item(l-1,p[0]-1)  
            lenghts.append(lenght)
        minLenght=1000   
        for l in lenghts:
            if l<minLenght:
                minLenght=l    
        #Обновление феормона на путях для элитных муравьев
        for m in paths[lenghts.index(minLenght)]:
            counter+=1
            if (counter)<9:
                np.add.at(pheromons,(m-1,p[counter+1]-1),Q/minLenght)
                np.add.at(pheromons,(p[counter+1]-1,m-1),Q/minLenght)
            else:
                np.add.at(pheromons,(p[0]-1,m-1),Q/minLenght)
                np.add.at(pheromons,(m-1,p[0]-1),Q/minLenght)
#         print("Феромоны: ",pheromons)
#         print(n)
        
#         print(pheromons)
    
        if minLenght<200:
            break
            
    end=time.perf_counter()
    T=(end-start)
    print("T= ",T,"c")        
    print(minLenght)
    print(lenghts)
    for p in paths:
        print(p)
    
                                   
ANT(10,10000)  

Процесс начат
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1]
[0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1]
[0.1, 0.1]
[0.1, 0.1, 0.1, 0

[3.4246576342465815e-37]
[29.676438462052463, 32.347418385310455, 29.6736218754712, 29.673621560644012, 29.709853444702016, 29.673926438695368, 29.982266329248745, 29.673621560644012]
[0.3114616700131824, 0.30864508343191716, 0.3086447686047314, 0.3448766526627358, 0.3089496466560892, 0.3086447686047314]
[0.0031217794598087964, 0.00030519287854353114, 0.0003048780513578075, 0.0003048780513578075]
[0.0028172162356367124, 0.002816901408450989, 0.002816901408450989]
[3.4246576342465815e-37]
[32.347418385310455, 29.6736218754712, 29.673621560644012, 29.709853444702016, 29.673926438695368, 29.673621560644012, 29.982266329248745, 29.673621560644012]
[2.67379713949363, 2.6737968246664443, 2.710028708724449, 2.6741017027178025, 2.6737968246664443, 2.982441593271176, 2.6737968246664443]
[0.30864508343191716, 0.3086447686047314, 0.3448766526627358, 0.3089496466560892, 0.3086447686047314, 0.3086447686047314]
[0.00030519287854353114, 0.0003048780513578075, 0.0003048780513578075]
[3.148271857236318

[3.4181083991678825e-11, 3.4180823572249715e-11]
[2.604194290976061e-16]
[0.3303088320343457, 0.3332080806941726, 0.33003590788156834, 0.33003303180861465, 0.330033031503525, 0.3300330314693445, 29.3155402778462, 4.094379615381467, 0.3300330314693442]
[28.985783046941858, 28.988682295601684, 28.98551012278908, 28.985507246716125, 28.985507246411036, 28.985507246376855, 32.74985383028898, 28.985507246376855]
[3.7646223844771245, 3.767521633136951, 3.764349460324347, 3.7643465842513937, 3.7643465839463035, 3.7643465839121237, 3.764346583912123]
[0.0034508497898299256, 0.0031779256370525473, 0.0031750495640988706, 0.0031750492590092573, 0.003175049224828695, 0.0031750492248284337]
[2.876751494550153e-06, 3.7345126032737603e-10, 3.3927069717455537e-10, 3.392704367551263e-10]
[3.7646223844771245, 3.767521633136951, 3.764349460324347, 3.7643465842513937, 4.094379615381467, 3.7643465839463035, 3.7643465839121237, 32.74985383028898, 3.764346583912123]
[0.0034508497898299256, 0.0031750495640988

In [461]:
print(maps)
print(np.where(maps==np.amax(maps)))
maps1=np.delete(maps,5,1)
print(maps1)
print(np.where(maps1==np.amax(maps1)))

[[ 1 57 55  4 26 84 72 14 11 55]
 [57  1 73 65 38 97 41 13 39 57]
 [55 73  1 19 36 25 63  2 87 81]
 [ 4 65 19  1 46 60 80 21 41 84]
 [26 38 36 46  1 17  1 66 60 52]
 [84 97 25 60 17  1 50 97 88 45]
 [72 41 63 80  1 50  1  5 79 46]
 [14 13  2 21 66 97  5  1  3 94]
 [11 39 87 41 60 88 79  3  1 46]
 [55 57 81 84 52 45 46 94 46  1]]
(array([1, 5, 5, 7]), array([5, 1, 7, 5]))
[[ 1 57 55  4 26 72 14 11 55]
 [57  1 73 65 38 41 13 39 57]
 [55 73  1 19 36 63  2 87 81]
 [ 4 65 19  1 46 80 21 41 84]
 [26 38 36 46  1  1 66 60 52]
 [84 97 25 60 17 50 97 88 45]
 [72 41 63 80  1  1  5 79 46]
 [14 13  2 21 66  5  1  3 94]
 [11 39 87 41 60 79  3  1 46]
 [55 57 81 84 52 46 94 46  1]]
(array([5, 5]), array([1, 6]))


In [457]:
print(np.ones([10,10]))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
